# Saddleplots - plotting only !

In [ ]:
# import standard python libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os, subprocess

In [ ]:
# Import python package for working with cooler files and tools for analysis
import cooler
import cooltools.lib.plotting

In [ ]:
%load_ext autoreload
%autoreload 2
# from saddle import saddleplot

In [ ]:
# download test data
# this file is 145 Mb, and may take a few seconds to download
import bbi
import cooltools
import bioframe
from matplotlib.colors import LogNorm
from helper_func import saddleplot
from data_catalog import bws, bws_vlim, telo_dict

import saddle


In [ ]:
from tqdm import tqdm
from tqdm.notebook import trange, tqdm
import warnings
import seaborn as sns

import warnings
import h5py

import matplotlib.lines as lines
from matplotlib.lines import Line2D
from matplotlib.patches import ConnectionPatch, Rectangle
from mpl_toolkits.axes_grid1 import Divider, Size
from mpl_toolkits.axes_grid1.inset_locator import BboxConnector
from matplotlib import cm
import matplotlib as mpl
# from mpl_toolkits.axes_grid1.Size import Fixed


# enable editable text ...
mpl.rcParams["pdf.fonttype"]=42
mpl.rcParams["svg.fonttype"]="none"
mpl.rcParams['axes.linewidth'] = 0.5

## Calculating per-chromosome compartmentalization

We first load the Hi-C data at 100 kbp resolution. 

Note that the current implementation of eigendecomposition in cooltools assumes that individual regions can be held in memory-- for hg38 at 100kb this is either a 2422x2422 matrix for chr2, or a 3255x3255 matrix for the full cooler here.

In [ ]:
# define genomic view that will be used to call dots and pre-compute expected

# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
hg38_cens = bioframe.fetch_centromeres('hg38')
hg38_arms_full = bioframe.make_chromarms(hg38_chromsizes, hg38_cens)
# # remove "bad" chromosomes and near-empty arms ...
included_arms = hg38_arms_full["name"].to_list()[:44] # all autosomal ones ...
hg38_arms = hg38_arms_full[hg38_arms_full["name"].isin(included_arms)].reset_index(drop=True)

In [ ]:
! ls *.hdf5

In [ ]:

# fr.items()
def print_attrs(name, obj):
    # Create indent
    shift = name.count('/') * '    '
    item_name = name.split("/")[-1]
    print(shift + item_name)
    try:
        for key, val in obj.attrs.items():
            print(shift + '    ' + f"{key}: {val}")
    except:
        pass

# load IPG saddles asis ...
print("loading IPG saddles as is ...")
print()
with h5py.File("saddles_IPG_by_distance_asis.hdf5", 'r') as fr:
# with h5py.File("saddles_cre_by_distance_Allana_latest_cre.hdf5", 'r') as fr:
    # f = h5py.File('foo.hdf5','r')
    fr.visititems(print_attrs)

    # check general metadata ...
    _saddle_meta = dict(fr.attrs)

    # sort out the results ...
    interaction_sums_trans_asis = {}
    interaction_counts_trans_asis = {}
    # sort out the results ...
    _counts = fr.get("counts_trans")
    for _sample in _counts.keys():
        interaction_counts_trans_asis[_sample] = _counts.get(_sample)[()]
    _sums = fr.get("sums_trans")
    for _sample in _sums.keys():
        interaction_sums_trans_asis[_sample] = _sums.get(_sample)[()]

    interaction_sums_asis = {}
    interaction_counts_asis = {}
    # sort out the results ...
    _counts = fr.get("counts")
    for _sample in _counts.keys():
        interaction_counts_asis[_sample] = _counts.get(_sample)[()]
    _sums = fr.get("sums")
    for _sample in _sums.keys():
        interaction_sums_asis[_sample] = _sums.get(_sample)[()]


# load IPG saddles with IDs ...
print("loading IPG saddles with IDs ...")
print()
with h5py.File("saddles_IPG_by_distance_wIDs.hdf5", 'r') as fr:
# with h5py.File("saddles_cre_by_distance_Allana_latest_cre.hdf5", 'r') as fr:
    # f = h5py.File('foo.hdf5','r')
    fr.visititems(print_attrs)

    # check general metadata ...
    _saddle_meta = dict(fr.attrs)

    # sort out the results ...
    interaction_sums_trans_wids = {}
    interaction_counts_trans_wids = {}
    # sort out the results ...
    _counts = fr.get("counts_trans")
    for _sample in _counts.keys():
        interaction_counts_trans_wids[_sample] = _counts.get(_sample)[()]
    _sums = fr.get("sums_trans")
    for _sample in _sums.keys():
        interaction_sums_trans_wids[_sample] = _sums.get(_sample)[()]

    interaction_sums_wids = {}
    interaction_counts_wids = {}
    # sort out the results ...
    _counts = fr.get("counts")
    for _sample in _counts.keys():
        interaction_counts_wids[_sample] = _counts.get(_sample)[()]
    _sums = fr.get("sums")
    for _sample in _sums.keys():
        interaction_sums_wids[_sample] = _sums.get(_sample)[()]


In [ ]:
sub_samples_m = [
    "mCyto",
    "m5hR1R2",
]
sub_samples_p = [
    "pCyto",
    "p5hR1R2",
]

# introduce distance ranges
distances = {
    "cis": slice(None),
    "trans": slice(None),
}


# cbarw = 0.7*matw
margin = 0.2
matw = 0.75
cbarh = 0.1

imshow_kwargs = dict(
        norm=LogNorm(vmin=1/3, vmax=3),
        cmap="RdBu_r",
        interpolation="nearest",
)


legend_kwargs = dict(
    aspect="auto",
    interpolation="nearest",
)

# create legends ......
IPG_cmap = {
    3: "cornflowerblue",  # B
    2: "#ffee99",  # V+VI
    1: "orangered",  # A2
    0: "maroon",  # A1
    # 0: "#D9E2EF",  #"none"
}
ticklabels_ipg=["B","VVI","A2","A1"]


IPGwID_cmap = {
    6: "cornflowerblue",  # B
    5: "#ffee99",  # V+VI
    4: "#ffee99",  # V+VI-ID
    3: "orangered",  # A2
    2: "orangered",  # A2-ID
    1: "maroon",  # A1
    0: "maroon",  # A1-ID
    # 0: "#D9E2EF",  #"none"
}
ticklabels_ipgid=["B","VVI","VVI-ID","A2","A2-ID","A1","A1-ID"]


## Draw actual figures with the semi-manual custom layout ...

In [ ]:

h = [
    Size.Fixed(margin),
    # Cyto
    Size.Fixed(matw),
    Size.Fixed(0.2*margin),
    Size.Fixed(matw),
    Size.Fixed(margin),
    # 5hr
    Size.Fixed(matw),
    Size.Fixed(0.2*margin),
    Size.Fixed(matw),
    # bar ...
    Size.Fixed(0.2*margin),
    Size.Fixed(cbarh),
    Size.Fixed(margin),
]

# goes from bottom to the top ...
v = [
    # single color bar at the vewry bottom
    Size.Fixed(margin),
    Size.Fixed(0.5*cbarh),
    # bottom - with IDs
    Size.Fixed(0.5*margin),
    Size.Fixed(cbarh),
    Size.Fixed(0.2*margin),
    Size.Fixed(matw),
    Size.Fixed(0.2*margin),
    Size.Fixed(matw),
    # upper - as is ...
    Size.Fixed(margin),
    Size.Fixed(cbarh),
    Size.Fixed(0.2*margin),
    Size.Fixed(matw),
    Size.Fixed(0.2*margin),
    Size.Fixed(matw),
    Size.Fixed(margin),
]
_stickingout_bit = sum(_h.fixed_size for _h in h[-3:])
print(f"{_stickingout_bit=}")

# set figsize based on the tiling provided ...
fig_width = sum(_h.fixed_size for _h in h)
fig_height = sum(_v.fixed_size for _v in v)
fig = plt.figure(
    figsize=(fig_width, fig_height),
    # facecolor='lightblue'
)
print(f"figure size {fig_width=} {fig_height=}")
# ...
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)



axs = {}
for i, _sample in enumerate(sub_samples_m + sub_samples_p):
    axs[_sample] = {}
    _nx_i = 2*i+1
    axs[_sample]["bottom_legend"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=1+2))
    axs[_sample]["trans_ID_saddle"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=3+2))
    axs[_sample]["cis_ID_saddle"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=5+2))
    axs[_sample]["upper_legend"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=7+2))
    axs[_sample]["trans_IPG_saddle"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=9+2))
    axs[_sample]["cis_IPG_saddle"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=11+2))
# colorbar ...
cbar_ax = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=1))
# very last column with legends ....
_nx_i = 2*(i+1)+1
axs["bottom_right_legend"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=3+2))
axs["bottom_right_legend2"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=5+2))
axs["upper_right_legend"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=9+2))
axs["upper_right_legend2"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx_i, ny=11+2))

# traverse nested dict to access the axes ...
for _ax in sum([ [v for v in axs[s].values()] for s in (sub_samples_m+sub_samples_p) ], start=[]):
    _ax.set_xticks([])
    _ax.set_yticks([])
# do the remaining ones (not nested one as well) ...
for _ax in [ v for k,v in axs.items() if k not in (sub_samples_m+sub_samples_p) ]:
    _ax.set_xticks([])
    _ax.set_yticks([])
cbar_ax.set_xticks([])
cbar_ax.set_yticks([])

# reorder for the ones with IDs ...
_reidxs = [0,3,1,4,2,5,6]


for _sample in (sub_samples_m+sub_samples_p):
    for _dist_key, _dist in distances.items():
        # IPG saddles As is ...
        if _dist_key != "trans":
            C = np.nanmean(interaction_sums_asis[_sample][_dist], axis=0) / np.nanmean(interaction_counts_asis[_sample][_dist], axis=0)
        elif _dist_key == "trans":
            C = np.nanmean(interaction_sums_trans_asis[_sample][_dist], axis=0) / np.nanmean(interaction_counts_trans_asis[_sample][_dist], axis=0)
        else:
            pass
        _h = axs[_sample][f"{_dist_key}_IPG_saddle"].imshow(C[1:,1:], **imshow_kwargs)
        _h.cmap.set_over("#300000")
        # extract size ...
        _num_ipg_colors, _ = C[1:,1:].shape
        # IPG saddles with IDs ...
        if _dist_key != "trans":
            C = np.nanmean(interaction_sums_wids[_sample][_dist], axis=0) / np.nanmean(interaction_counts_wids[_sample][_dist], axis=0)
        elif _dist_key == "trans":
            C = np.nanmean(interaction_sums_trans_wids[_sample][_dist], axis=0) / np.nanmean(interaction_counts_trans_wids[_sample][_dist], axis=0)
        else:
            pass
        _h = axs[_sample][f"{_dist_key}_ID_saddle"].imshow(C[1:,1:][_reidxs][:,_reidxs], **imshow_kwargs)
        _h.cmap.set_over("#300000")
        # extract size ...
        _num_ipg_wIDs_colors, _ = C[1:,1:].shape

# create a "fake" legend for IPGs - for now ...
_fdata = np.reshape(np.arange(_num_ipg_colors), (-1,1))
_fcmap = plt.cm.gray
_fnorm = plt.Normalize()
_frgba = _fcmap(_fnorm(_fdata))
_frgbaT = _fcmap(_fnorm(_fdata.T))
# ...
for i in range(_num_ipg_colors):
    _frgba[i,0] = list(mpl.colors.to_rgb(IPG_cmap[i]))+[1]
    _frgbaT[0,i] = list(mpl.colors.to_rgb(IPG_cmap[i]))+[1]

for _sample in (sub_samples_m+sub_samples_p):
    axs[_sample]["upper_legend"].imshow(_frgbaT, **legend_kwargs)
    for _ in range(max(IPG_cmap.keys())):
        axs[_sample]["upper_legend"].axvline(_+.5,color="black",lw=0.5)
axs["upper_right_legend"].imshow(_frgba, **legend_kwargs)
axs["upper_right_legend2"].imshow(_frgba, **legend_kwargs)
for _ in range(max(IPG_cmap.keys())):
    axs["upper_right_legend"].axhline(_+.5,color="black",lw=0.5)
    axs["upper_right_legend2"].axhline(_+.5,color="black",lw=0.5)
# try adding labels to the right ...
axs["upper_right_legend"].yaxis.tick_right()
axs["upper_right_legend"].set_yticks(
    list(IPG_cmap.keys()),
    labels=ticklabels_ipg,
    fontsize=6,
)
axs["upper_right_legend"].tick_params(length=1.5, pad=1)

# create a "fake" legend for IPGs - for now ...
_fdata = np.reshape(np.arange(_num_ipg_wIDs_colors), (-1,1))
_frgba = _fcmap(_fnorm(_fdata))
_frgbaT = _fcmap(_fnorm(_fdata.T))
# ...
for i in range(_num_ipg_wIDs_colors):
    _frgba[i,0] = list(mpl.colors.to_rgb(IPGwID_cmap[i]))+[1]
    _frgbaT[0,i] = list(mpl.colors.to_rgb(IPGwID_cmap[i]))+[1]

for _sample in (sub_samples_m+sub_samples_p):
    axs[_sample]["bottom_legend"].imshow(_frgbaT, **legend_kwargs)
    for _ in range(max(IPGwID_cmap.keys())):
        axs[_sample]["bottom_legend"].axvline(_+.5,color="black",lw=0.5)
axs["bottom_right_legend"].imshow(_frgba, **legend_kwargs)
axs["bottom_right_legend2"].imshow(_frgba, **legend_kwargs)
for _ in range(max(IPGwID_cmap.keys())):
    axs["bottom_right_legend"].axhline(_+.5,color="black",lw=0.5)
    axs["bottom_right_legend2"].axhline(_+.5,color="black",lw=0.5)
# try adding labels to the right ...
axs["bottom_right_legend"].yaxis.tick_right()
axs["bottom_right_legend"].set_yticks(
    list(IPGwID_cmap.keys()),
    labels=ticklabels_ipgid,
    fontsize=6,
)
axs["bottom_right_legend"].tick_params(length=1.5, pad=1)


# add a single colorbar ...
fig.colorbar(
    cm.ScalarMappable(norm=imshow_kwargs["norm"], cmap=imshow_kwargs["cmap"]),
    cax=cbar_ax,
    orientation="horizontal",
)
_vmin = imshow_kwargs["norm"].vmin
_vmax = imshow_kwargs["norm"].vmax
cbar_ax.set_xticks([_vmin, 1, _vmax])
cbar_ax.set_xticklabels([f"{_vmin:.2f}", 1, _vmax], fontsize=6)
cbar_ax.minorticks_off()
cbar_ax.tick_params(length=1.5, pad=1)#,direction='out', length=6, width=2, colors='r', grid_color='r', grid_alpha=0.5)
for _tidx, tick in enumerate(cbar_ax.xaxis.get_majorticklabels()):
    if _tidx == 0:
        tick.set_horizontalalignment("left")
    elif _tidx == 2:
        tick.set_horizontalalignment("right")
    else:
        tick.set_horizontalalignment("center")

fig.savefig("Fig6D.svg", dpi=300)


In [ ]:

h = [
    Size.Fixed(margin),
    Size.Fixed(matw), #ctrl
    Size.Fixed(0.2*margin),
    Size.Fixed(matw),  # nup93
    Size.Fixed(0.2*margin),
    Size.Fixed(cbarh),
    Size.Fixed(margin),
]

# goes from bottom to the top ...
v = [
    # single color bar at the vewry bottom
    Size.Fixed(margin),
    Size.Fixed(0.5*cbarh),
    Size.Fixed(0.33*margin),
    Size.Fixed(cbarh),
    Size.Fixed(0.2*margin),
    Size.Fixed(matw),
    Size.Fixed(0.2*margin),
    Size.Fixed(matw),
    Size.Fixed(margin),
]
_stickingout_bit = sum(_h.fixed_size for _h in h[-3:])
print(f"{_stickingout_bit=}")

# set figsize based on the tiling provided ...
fig_width = sum(_h.fixed_size for _h in h)
fig_height = sum(_v.fixed_size for _v in v)
fig = plt.figure(
    figsize=(fig_width, fig_height),
    # facecolor='lightblue'
)
print(f"figure size {fig_width=} {fig_height=}")
# ...
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)


nup_samples = ["N93m5","N93p5"]

axs = {}
axl = {}
axv = {}
for i, _sample in enumerate(nup_samples):
    _nx = 2*i+1
    axs[_sample] = {}
    axl[_sample] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx, ny=3))
    for j, _dist_key in enumerate(reversed(distances)):
        _ny = 2*j+5
        axs[_sample][_dist_key] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx, ny=_ny))
        if _sample == nup_samples[-1]:
            axv[_dist_key] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx+2, ny=_ny))
# colorbar ...
cbar_ax = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx, ny=1))

# traverse nested dict to access the axes ...
for _ax in (
    sum([ [v for v in axs[s].values()] for s in nup_samples ], start=[]) +
    list(axl.values()) + list(axv.values()) + [cbar_ax]
):
    _ax.set_xticks([])
    _ax.set_yticks([])

# reorder for the ones with IDs ...
_reidxs = [0,3,1,4,2,5,6]
for _sample in nup_samples:
    for _dist_key, _dist in distances.items():
        # IPG saddles w IDs ....
        if _dist_key != "trans":
            C = np.nanmean(interaction_sums_wids[_sample][_dist], axis=0) / np.nanmean(interaction_counts_wids[_sample][_dist], axis=0)
        elif _dist_key == "trans":
            C = np.nanmean(interaction_sums_trans_wids[_sample][_dist], axis=0) / np.nanmean(interaction_counts_trans_wids[_sample][_dist], axis=0)
        else:
            pass
        _h = axs[_sample][_dist_key].imshow(C[1:,1:][_reidxs][:,_reidxs], **imshow_kwargs)
        _h.cmap.set_over("#300000")
        # ...
        if _sample == nup_samples[0]:
            axs[_sample][_dist_key].set_ylabel(_dist_key, fontsize=8, labelpad=1)
        if _dist_key == "cis":
            axs[_sample][_dist_key].set_title(_sample, fontsize=8, pad=1)
        # extract size ...
        _num_ipg_wIDs_colors, _ = C[1:,1:].shape

# create a "fake" legend for IPGs - for now ...
_fdata = np.reshape(np.arange(_num_ipg_wIDs_colors), (-1,1))
_fcmap = plt.cm.gray
_fnorm = plt.Normalize()
_frgba = _fcmap(_fnorm(_fdata))
_frgbaT = _fcmap(_fnorm(_fdata.T))
# ...
for i in range(_num_ipg_wIDs_colors):
    _frgba[i,0] = list(mpl.colors.to_rgb(IPGwID_cmap[i]))+[1]
    _frgbaT[0,i] = list(mpl.colors.to_rgb(IPGwID_cmap[i]))+[1]


for _sample in nup_samples:
    axl[_sample].imshow(_frgbaT, **legend_kwargs)
    for _ in range(max(IPGwID_cmap.keys())):
        axl[_sample].axvline(_+.5,color="black",lw=0.5)

for _dist_key in distances:
    axv[_dist_key].imshow(_frgba, **legend_kwargs)
    for _ in range(max(IPGwID_cmap.keys())):
        axv[_dist_key].axhline(_+.5,color="black",lw=0.5)

# try adding labels to the right ...
axv["trans"].yaxis.tick_right()
axv["trans"].set_yticks(
    list(IPGwID_cmap.keys()),
    labels=ticklabels_ipgid,
    fontsize=6,
)
axv["trans"].tick_params(length=1.5, pad=1)


# add a single colorbar ...
fig.colorbar(
    cm.ScalarMappable(norm=imshow_kwargs["norm"], cmap=imshow_kwargs["cmap"]),
    cax=cbar_ax,
    orientation="horizontal",
)
_vmin = imshow_kwargs["norm"].vmin
_vmax = imshow_kwargs["norm"].vmax
cbar_ax.set_xticks([_vmin, 1, _vmax])
cbar_ax.set_xticklabels([f"{_vmin:.2f}", 1, _vmax], fontsize=6)
cbar_ax.minorticks_off()
cbar_ax.tick_params(length=1.5, pad=1)#,direction='out', length=6, width=2, colors='r', grid_color='r', grid_alpha=0.5)
for _tidx, tick in enumerate(cbar_ax.xaxis.get_majorticklabels()):
    if _tidx == 0:
        tick.set_horizontalalignment("left")
    elif _tidx == 2:
        tick.set_horizontalalignment("right")
    else:
        tick.set_horizontalalignment("center")

fig.savefig("FigExt76E_nup.svg", dpi=300)


# Older stuff that isn't publication ready ...

In [ ]:
fig, axs = plt.subplots(
    nrows=len(sub_samples_m),
    ncols=2*len(distances),
    figsize=(4*len(distances),2*len(sub_samples_m)),
    sharex=True,
    sharey=True,
)

imshow_kwargs = dict(
        norm=LogNorm(vmin=1/2, vmax=2),
        cmap="RdBu_r",
        interpolation="none",
)

for sample_m, sample_p, (i, axs) in zip(sub_samples_m, sub_samples_p, enumerate(axs)):
    for jj, (_dist_name, _dist) in enumerate(distances.items()):
        axm, axp = axs[jj], axs[len(distances) + jj]
        if _dist_name != "trans":
            Cm = np.nanmean(interaction_sums[sample_m][_dist], axis=0) / np.nanmean(interaction_counts[sample_m][_dist], axis=0)
            Cp = np.nanmean(interaction_sums[sample_p][_dist], axis=0) / np.nanmean(interaction_counts[sample_p][_dist], axis=0)
        elif _dist_name == "trans":
            # pass
            Cm = np.nanmean(interaction_sums_trans[sample_m][_dist], axis=0) / np.nanmean(interaction_counts_trans[sample_m][_dist], axis=0)
            Cp = np.nanmean(interaction_sums_trans[sample_p][_dist], axis=0) / np.nanmean(interaction_counts_trans[sample_p][_dist], axis=0)
        else:
            pass
        axm.imshow(Cm, **imshow_kwargs)
        axp.imshow(Cp, **imshow_kwargs)
        for _ax in [axp, axm]:
            _ax.set_xticks([])
            _ax.set_yticks([])
        if i == 0:
            axm.set_title(f"m-{_dist_name}")
            axp.set_title(f"p-{_dist_name}")
        if i == len(sub_samples_m)-1:
            for _ax in [axm, axp]:
                _ax.set_xticks(np.arange(len(ticklabels)))
                _ax.set_xticklabels(np.asarray(ticklabels[::-1]), rotation="vertical")
        if jj == 0:
            axm.set_ylabel(sample_m.lstrip("m"))



In [ ]:
fig, axs = plt.subplots(
    nrows=len(sub_samples_m),
    ncols=2*len(distances),
    figsize=(4*len(distances),2*len(sub_samples_m)),
    sharex=True,
    sharey=True,
)

imshow_kwargs = dict(
        norm=LogNorm(vmin=1/2.25, vmax=2.25),
        cmap="RdBu_r",
        interpolation="none",
)

for sample_m, sample_p, (i, axs) in zip(sub_samples_m, sub_samples_p, enumerate(axs)):
    for jj, (_dist_name, _dist) in enumerate(distances.items()):
        axm, axp = axs[jj], axs[len(distances) + jj]
        if _dist_name != "trans":
            Cm = np.nanmean(interaction_sums[sample_m][_dist], axis=0) / np.nanmean(interaction_counts[sample_m][_dist], axis=0)
            Cp = np.nanmean(interaction_sums[sample_p][_dist], axis=0) / np.nanmean(interaction_counts[sample_p][_dist], axis=0)
        elif _dist_name == "trans":
            # pass
            Cm = np.nanmean(interaction_sums_trans[sample_m][_dist], axis=0) / np.nanmean(interaction_counts_trans[sample_m][_dist], axis=0)
            Cp = np.nanmean(interaction_sums_trans[sample_p][_dist], axis=0) / np.nanmean(interaction_counts_trans[sample_p][_dist], axis=0)
        else:
            pass
        axm.imshow(Cm, **imshow_kwargs)
        axp.imshow(Cp, **imshow_kwargs)
        for _ax in [axp, axm]:
            _ax.set_xticks([])
            _ax.set_yticks([])
        if i == 0:
            axm.set_title(f"m-{_dist_name}")
            axp.set_title(f"p-{_dist_name}")
        if i == len(sub_samples_m)-1:
            for _ax in [axm, axp]:
                _ax.set_xticks(np.arange(len(ticklabels)))
                _ax.set_xticklabels(np.asarray(ticklabels[::-1]), rotation="vertical")
        if jj == 0:
            axm.set_ylabel(sample_m.lstrip("m"))

In [ ]:
# # # # the mix one - mp
sub_samples_m = [
    "N93m5",
    "N93m10",
]
# p ...
sub_samples_p = [
    "N93p5",
    "N93p10",
]

fig, axs = plt.subplots(
    nrows=len(sub_samples_m),
    ncols=2*len(distances),
    figsize=(4*len(distances),2*len(sub_samples_m)),
    sharex=True,
    sharey=True,
)

imshow_kwargs = dict(
        norm=LogNorm(vmin=1/2.5, vmax=2.5),
        cmap="RdBu_r",
        interpolation="none",
)

for sample_m, sample_p, (i, axs) in zip(sub_samples_m, sub_samples_p, enumerate(axs)):
    for jj, (_dist_name, _dist) in enumerate(distances.items()):
        axm, axp = axs[jj], axs[len(distances) + jj]
        if _dist_name != "trans":
            Cm = np.nanmean(interaction_sums[sample_m][_dist], axis=0) / np.nanmean(interaction_counts[sample_m][_dist], axis=0)
            Cp = np.nanmean(interaction_sums[sample_p][_dist], axis=0) / np.nanmean(interaction_counts[sample_p][_dist], axis=0)
        elif _dist_name == "trans":
            Cm = np.nanmean(interaction_sums_trans[sample_m][_dist], axis=0) / np.nanmean(interaction_counts_trans[sample_m][_dist], axis=0)
            Cp = np.nanmean(interaction_sums_trans[sample_p][_dist], axis=0) / np.nanmean(interaction_counts_trans[sample_p][_dist], axis=0)
        else:
            pass
        axm.imshow(Cm, **imshow_kwargs)
        axp.imshow(Cp, **imshow_kwargs)
        for _ax in [axp, axm]:
            _ax.set_xticks([])
            _ax.set_yticks([])
        if i == 0:
            axm.set_title(f"m-{_dist_name}")
            axp.set_title(f"p-{_dist_name}")
        if i == len(sub_samples_m)-1:
            for _ax in [axm, axp]:
                _ax.set_xticks(np.arange(len(ticklabels)))
                _ax.set_xticklabels(np.asarray(ticklabels[::-1]), rotation="vertical")
        if jj == 0:
            axm.set_ylabel(sample_m.lstrip("m"))

In [ ]:
sub_samples_m =[
        "m10hR1R2",
        "p10hR1R2",
        "mp10hR1R2",
    ]
sub_samples_p = [
        "N93m10",
        "N93p10",
        "N93mp10",
    ]

fig, axs = plt.subplots(
    nrows=len(sub_samples_m),
    ncols=2*len(distances),
    figsize=(4*len(distances),2*len(sub_samples_m)),
    sharex=True,
    sharey=True,
)

imshow_kwargs = dict(
        norm=LogNorm(vmin=1/2.5, vmax=2.5),
        cmap="RdBu_r",
        interpolation="none",
)

for sample_m, sample_p, (i, axs) in zip(sub_samples_m, sub_samples_p, enumerate(axs)):
    for jj, (_dist_name, _dist) in enumerate(distances.items()):
        axm, axp = axs[jj], axs[len(distances) + jj]
        if _dist_name != "trans":
            Cm = np.nanmean(interaction_sums[sample_m][_dist], axis=0) / np.nanmean(interaction_counts[sample_m][_dist], axis=0)
            Cp = np.nanmean(interaction_sums[sample_p][_dist], axis=0) / np.nanmean(interaction_counts[sample_p][_dist], axis=0)
        elif _dist_name == "trans":
            Cm = np.nanmean(interaction_sums_trans[sample_m][_dist], axis=0) / np.nanmean(interaction_counts_trans[sample_m][_dist], axis=0)
            Cp = np.nanmean(interaction_sums_trans[sample_p][_dist], axis=0) / np.nanmean(interaction_counts_trans[sample_p][_dist], axis=0)
        else:
            pass
        axm.imshow(Cm, **imshow_kwargs)
        axp.imshow(Cp, **imshow_kwargs)
        for _ax in [axp, axm]:
            _ax.set_xticks([])
            _ax.set_yticks([])
        if i == 0:
            axm.set_title(f"m-{_dist_name}")
            axp.set_title(f"p-{_dist_name}")
        if i == len(sub_samples_m)-1:
            for _ax in [axm, axp]:
                _ax.set_xticks(np.arange(len(ticklabels)))
                _ax.set_xticklabels(np.asarray(ticklabels[::-1]), rotation="vertical")
        if jj == 0:
            axm.set_ylabel(sample_m)



# try adding an axes manually ...
cax = fig.add_axes([0.88,0.001,0.1,0.02])
fig.colorbar(
    cm.ScalarMappable(norm=imshow_kwargs["norm"], cmap=imshow_kwargs["cmap"]),
    cax=cax,
    orientation="horizontal",
)
cax.set_xticks([imshow_kwargs["norm"].vmin, 1, imshow_kwargs["norm"].vmax])
cax.set_xticklabels([imshow_kwargs["norm"].vmin, 1, imshow_kwargs["norm"].vmax])
cax.minorticks_off()